In [138]:
from nltk.tokenize import RegexpTokenizer

In [70]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import numpy as np
import pandas as pd
import gensim

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import os

In [95]:
support_or_not=pd.read_csv('../output/data_with_vadersentiment.csv',encoding="utf-8")

In [96]:
support_or_not.head()

,Unnamed: 0,Unnamed: 0.1,id,text,user_id,user_name,user_followers,user_loc,lang,retweet_id,...,retweet_status_user_name,retweet_status_follower_count,retweet_status_favorite_count,retweet_status_retweet_count,retweet_status_reply_count,created_at,neg,neu,pos,compound
0,0,0,980822559901016070,Here ’ full list American goods affected China ’ retaliatory tariffs #create,26558316,blackwhalegroup,335,NYC SD KC,en,NaN,...,NaN,NaN,NaN,NaN,NaN,2018-04-02 15:01:35,0.151,0.849,0.00,-0.1531
1,1,1,980822563679911937,Hope he's ready give aid farmers He'll probably wait Willy Nelson others hold concerts …,1525168716,Donna,457,california,en,NaN,...,NaN,NaN,NaN,NaN,NaN,2018-04-02 15:01:36,0.000,0.690,0.31,0.6597
2,2,2,980822565378838531,RT @RedTRaccoon NC pay high price metals tariffs North Carolina also get hit massive 25 tariff pork As stat …,1148610157,American AntiFed,16431,New Mexico,en,9.808189e+17,...,Red T Raccoon,97822.0,9.808189e+17,30.0,5.0,2018-04-02 15:01:37,0.072,0.928,0.00,-0.1027
3,3,3,980822567316590593,RT @MelissaLeeCNBC Tariff impact data already THERE WERE A LOT OF GENERAL CONCERNS ABOUT TARIFFS IN U S FACTORY ACTIVITY REPORT IN M …,601305639,Don Juan de Morocco,196,Internet,en,9.808139e+17,...,Melissa Lee,87491.0,9.808139e+17,3.0,1.0,2018-04-02 15:01:37,0.000,1.000,0.00,0.0000
4,4,4,980822568281235456,The full list 128 US products targeted China ’ retaliatory tariffs @qz @realDonaldTrump @GOP @SenateGOP,2279225400,Gloria (Dem) 🌊🇺🇸🌊,14158,New York USA,en,NaN,...,NaN,NaN,NaN,NaN,NaN,2018-04-02 15:01:37,0.000,1.000,0.00,0.0000


In [94]:
pd.set_option('display.max_colwidth', -1)

In [139]:
support_or_not.text[1]

"Hope he's ready give aid farmers He'll probably wait Willy Nelson others hold concerts"

# clean stop words like @ and other stuff

In [145]:
from nltk.tokenize import word_tokenize
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
#     print(tokens_re.search(tokens[0]))
    if lowercase:
        tokens =  [token if emoticon_re.search(token)  else token.lower() for token in tokens]
    return tokens
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'a n', 'example', '!', ':D', 'http://example.com', '#NLP']

['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']


['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']


In [127]:
from nltk.corpus import stopwords
import string
def remove_stop_words(tweet):
    out=tweet
    if(tweet):
        p=re.compile(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
        p2=re.compile(r'(?:@[\w_]+)')
        punctuation = list(string.punctuation)
        stop = stopwords.words('english') + punctuation + ['via',"\r",r'<[^>]+>',"RT","…","’","—","u"]+[r'(?:@[\w_]+)']
    #     print(stop)
        terms_stop = [term for term in preprocess(tweet) if p.match(term)==None and term not in stop and p2.match(term)==None]
    #     terms_stop = [term for term in preprocess(tweet) if term not in stop]
        out=" ".join(terms_stop)
    return(out)

In [128]:
remove_stop_words(support_or_not["text"][11])

'Here full list American goods affected China retaliatory tariffs'

In [129]:
support_or_not["text"][11]

'Here ’ full list American goods affected China ’ retaliatory tariffs'

In [130]:
mytext=list(map(lambda tweet: remove_stop_words(tweet), support_or_not.text))
support_or_not.text=mytext

# LDA analysis


In [114]:
pos_threshold=0.1
neg_threshold=-0.1

In [131]:
positive_list=list(support_or_not.loc[support_or_not["compound"]>=pos_threshold].text)

In [132]:
negative_list=list(support_or_not.loc[support_or_not["compound"]<=neg_threshold].text)

In [141]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
# doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
# doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
# doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
# doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
# doc_e = "Health professionals say that brocolli is good for your health." 

# compile sample documents into a list
# doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]
doc_set=positive_list
# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [140]:
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary, passes=20)

In [142]:
print(ldamodel.print_topics(num_topics=10, num_words=11))

[(0, '0.024*"tariff" + 0.020*"trade" + 0.018*"trump" + 0.014*"china" + 0.014*"s" + 0.011*"war" + 0.011*"increas" + 0.007*"win" + 0.007*"agricultur" + 0.007*"presid" + 0.007*"good"'), (1, '0.076*"2" + 0.075*"car" + 0.074*"sent" + 0.054*"state" + 0.050*"unit" + 0.043*"1" + 0.043*"china" + 0.042*"tariff" + 0.037*"paid" + 0.037*"trump" + 0.037*"chin"')]


In [133]:
positive_list[1]

'China hits United States tariffs 3 billion exports'

In [134]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [135]:
def topic(list_document,num_topics):
    """list_document is a list of string , each element is a document"""
    num_topics=num_topics
    doc_clean = [clean(doc).split() for doc in list_document] 
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    Lda = gensim.models.ldamodel.LdaModel
    ldamodel = Lda(doc_term_matrix, num_topics=num_topics, id2word = dictionary, passes=50)
    mylist=[]
    for topic in range(num_topics):
        termlist=ldamodel.print_topics(num_topics=num_topics, num_words=10)[topic][1].split("+")
        mylist.append(list(map(lambda term:term.split("*"),termlist)))
    pandas_list=[]
    for topic_index in range(num_topics):
        this_topic=pd.DataFrame(mylist[topic_index],columns=["prob","key_words"])
        this_topic["topic"]=topic_index
        pandas_list.append(this_topic)
    LDA_result=pd.concat(pandas_list)
    return(LDA_result)

In [148]:
positive_topic_dataframe=topic(positive_list,num_topics=20)

In [149]:
negative_topic_dataferame=topic(negative_list,num_topics=20)

In [185]:
negative_topic_dataferame.to_csv("negative_topics.csv")

In [186]:
positive_topic_dataframe.to_csv("positive_topics.csv")

In [188]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


In [190]:
def topic(list_document,num_topics):
    """list_document is a list of string , each element is a document"""
    num_topics=num_topics
    doc_clean = [clean(doc).split() for doc in list_document] 
    dictionary = corpora.Dictionary(doc_clean)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    Lda = gensim.models.ldamodel.LdaModel
    ldamodel = Lda(doc_term_matrix, num_topics=num_topics, id2word = dictionary, passes=50)
    return(ldamodel)

In [191]:
positive_model=topic(positive_list,num_topics=20)

In [198]:
path="../output/positve_model.pickle"

In [200]:
import pickle
with open(path, 'bw') as f:
    pickle.dump(positive_model, f)

In [195]:
for t in range(positive_model.num_topics):
    plt.figure()
    plt.imshow(WordCloud(background_color='white').fit_words(dict(positive_model.show_topic(t, 200))))
    plt.axis("off")
    plt.title("Positive Topic " )
    plt.savefig('../fig/positivefig'+str(t)+".png")

C:\Wesley_Tao\6.Software\Anaconda\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [196]:
negative_model=topic(negative_list,num_topics=20)

In [197]:
for t in range(negative_model.num_topics):
    plt.figure()
    plt.imshow(WordCloud(background_color='white').fit_words(dict(negative_model.show_topic(t, 200))))
    plt.axis("off")
    plt.title("Negative Topic" )
    plt.savefig('../fig/negativefig'+str(t)+".png")

C:\Wesley_Tao\6.Software\Anaconda\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [201]:
path="../output/negative_model.pickle"
with open(path, 'bw') as f:
    pickle.dump(negative_model, f)